In [ ]:
using Random
using Distributions
using Statistics
using Plots
using LinearAlgebra
using SpecialFunctions

# Scalings
Composite Trapezoidal Rule vs. Monte Carlo, as a function of dimension,
$$
\text{Composite Trapz Error}\propto \text{Work}^{-2/d}
$$
while
$$
\text{Monte Carlo Error}\propto \text{Work}^{-1/2}
$$
Here, work is measured in terms of number of $f$ evauations.

In [ ]:
work = 10 .^(1:10)
plot(work, work.^(-2), yscale=:log10, xscale=:log10, label="O(1/Work^2), Dimension 1",lw=2, xlabel="Work", ylabel="Error", legend=:bottomleft)
plot!(work, work.^(-1), yscale=:log10, xscale=:log10, label="O(1/Work^1), Dimension 2",lw=2)
plot!(work, work.^(-1/2), yscale=:log10, xscale=:log10, label="O(1/√Work), Monte Carlo",lw=2)
# plot!(work, work.^(-2/4), yscale=:log10, xscale=:log10, label="Dimension 4",lw=2)
plot!(work, work.^(-2/10), yscale=:log10, xscale=:log10, label="Dimension 10",lw=2)
plot!(work, work.^(-2/100), yscale=:log10, xscale=:log10, label="Dimension 100",lw=2)
plot!(xticks=10.0 .^(1:2:10))
plot!(yticks=10.0 .^(-16:2:0))

# Example
Estiamte
$$
\frac{2}{3}=\int_0^1 \sqrt{x}dx = \mathbb{E}[\sqrt{U}]\approx \frac{1}{n}\sum_{i=1}^n \sqrt{U_i}
$$
with $U_i \sim \mathrm{Unif}(0,1)$

In [ ]:
n = 10^8;
U = rand(n);
f(x) = sqrt(x);
@show mean(f.(U));

Empircally, using a larger `n` gives better results.

# Convergence Properties
How much does our estimate vary, if we repeated the procedure?  Repeat the experiment.

In [ ]:
n_trials = 10^4;
n = 10^2;
sample_means = [];
Random.seed!(1234); # for reproducibility
for i in 1:n_trials
    U = rand(n);
    push!(sample_means, mean(f.(U)));
end
histogram(sample_means, bins=20, label="Data", xlabel="Sample Means", ylabel="Frequency",
 title="Sample Means with n = $n")

Note the normality; this is the central limit theorem at work.

In [ ]:
@show mean(sample_means);
@show var(sample_means);

In [ ]:
n_trials = 10^4;
n = 10^4;
sample_means = [];
Random.seed!(1234); # for reproducibility
for i in 1:n_trials
    U = rand(n);
    push!(sample_means, mean(f.(U)));
end
histogram(sample_means, bins=20, label="Data", xlabel="Sample Means", ylabel="Frequency",
 title="Sample Means with n = $n")

In [ ]:
@show mean(sample_means);
@show var(sample_means);

In [ ]:
n_trials = 10^4;
n = 10^6;
sample_means = [];
Random.seed!(1234); # for reproducibility
for i in 1:n_trials
    U = rand(n);
    push!(sample_means, mean(f.(U)));
end
histogram(sample_means, bins=20, label="Data", xlabel="Sample Means", ylabel="Frequency",
 title="Sample Means with n = $n")

In [ ]:
@show mean(sample_means);
@show var(sample_means);

# An Estimate of Pi
Recall
$$
\int_0^1 \sqrt{1-x^2}dx = \frac{\pi}{4}
$$
So a Monte Carlo estimate of $\pi$ is given from
$$
\pi = \mathbb{E}[f(U)], \quad f(x) = 4 \sqrt{1-x^2}
$$

In [ ]:
n_trials = 10^4;
n = 10^4;
sample_means = [];
Random.seed!(1234); # for reproducibility

f(x) = 4*sqrt(1 - x^2);

for i in 1:n_trials
    U = rand(n);
    push!(sample_means, mean(f.(U)));
end
@show mean(sample_means);
@show var(sample_means);
histogram(sample_means, bins=20, label="Data", xlabel="Sample Means", ylabel="Frequency",
 title="Sample Means with n = $n")

# Another Esimate of Pi
Throw darts onto the $[-1,1]\times [-1,1]$ dart board, and estimate
$$
\pi = 4 \times \mathbb{E}[1(\|U\|_2 \leq 1)], \quad U \sim \mathrm{Unif}([-1,1]^2).
$$
To sample from this particular uniform distribution, we will use `Distributions` to construct the one we want.  We use `LinearAlgebra` to access the `norm` command.

In [ ]:
unif_dist = Uniform(-1,1); # defines the uniform distribution on [-1,1]
f(x) = norm(x) <= 1 ? 1 : 0

In [ ]:
n_trials = 10^4;
n = 10^4;
pi_estimates = [];
Random.seed!(1234); # for reproducibility

for i in 1:n_trials
    n_inside = 0;
    for j in 1:n
        U = rand(unif_dist, 2); # generate a dart in [-1,1] x [-1,1]
        n_inside += f(U);       # check if the dart is inside the unit circle
    end
    push!(pi_estimates, 4 * n_inside / n);
end


In [ ]:
@show mean(pi_estimates);
@show var(pi_estimates);
histogram(pi_estimates, bins=20, label="Data", xlabel="Sample Means", ylabel="Frequency",
 title="Pi Estimates with n = $n")

# Estimating the volume of a Unit Sphere in 3D
Recall, the answer is $4/3 \pi$

In [ ]:
@show 4/3 * π;

In [ ]:
n_trials = 10^4;
n = 10^4;
vol_estimates = [];
Random.seed!(1234); # for reproducibility

for i in 1:n_trials
    n_inside = 0;
    for j in 1:n
        U = rand(unif_dist, 3); # generate a dart in [-1,1] x [-1,1] x [-1,1]
        n_inside += f(U);       # check if the dart is inside the unit sphere
    end
    push!(vol_estimates, 2^3 * n_inside / n);
end

@show mean(vol_estimates);
@show var(vol_estimates);
histogram(vol_estimates, bins=20, label="Data", xlabel="Sample Means", ylabel="Frequency",
 title="Volume Estimates with n = $n")


# Volume of the Unit d-Sphere
$$
\frac{\pi^{d/2}}{\Gamma(\tfrac{d}{2}+1)}
$$
where $\Gamma$ is the gamma function.  We use `SpecialFunctions` to access this.

In [ ]:
d = 15;
n_trials = 10^4;
n = 10^4;
vol_estimates = [];
Random.seed!(1234); # for reproducibility

for i in 1:n_trials
    n_inside = 0;
    for j in 1:n
        U = rand(unif_dist, d); # generate a dart in [-1,1]^d
        n_inside += f(U);       # check if the dart is inside the unit sphere in d dimensions
    end
    push!(vol_estimates, 2^d * n_inside / n);
end

@show π^(d/2) / gamma(d/2 + 1);
@show mean(vol_estimates);
@show var(vol_estimates);
histogram(vol_estimates, bins=20, label="Data", xlabel="Sample Means", ylabel="Frequency",
 title="Volume with n = $n in dimension d = $d")
